In [1]:
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import os
import google.generativeai as genai
import fitz

printable = set(list(' ,`0123456789-=~!@#$%^&*()_+abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ,:;"\'/\n[]\\{}|?><.'))
bloat = {'per', 'and', 'but', 'the', 'for', 'are', 'was', 'were', 'be', 'been', 'with', 'you', 'this', 'but', 'his',
         'from', 'they', 'say', 'her', 'she', 'will', 'one', 'all', 'would', 'there', 'their', 'what', 'out', 'about',
         'who', 'get', 'which', 'when', 'make', 'can', 'like', 'time', 'just', 'into', 'year', 'your', 'good', 'some',
         'could', 'them', 'see', 'other', 'than', 'then', 'now', 'look', 'only', 'come', 'its', 'over', 'think', 'also',
         'back', 'after', 'use', 'two', 'how', 'our', 'work', 'first', 'well', 'way', 'even', 'new', 'want', 'because',
         'any', 'these', 'give', 'day', 'most'}
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

theta = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

C:\Users\aniru\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
def delimiter(phraset):
    l0 = 1
    for i in range(3):
        if phraset[i][1]>0.62:
            l0 += 1
    l1 = l0 + 1
    for i in range(10):
        if phraset[l0+i][1]>0.60:
            l1 += 1
    l2 = l1 + 1
    for i in range(20):
        if phraset[l1+i][1] >= 0.52:
            l2 += 1

    i1 = [x[0] for x in phraset[:l0]]
    i2 = [x[0] for x in phraset[l0:l1]]
    i3 = [x[0] for x in phraset[l1:l2]]
    return [i1, i2, i3]


def phrases_by_relevance(text, prompts):
    qembed = theta.encode([*prompts])
    embeddings = theta.encode(text)
    result = [tensor.item() for tensor in list(cos_sim(qembed, embeddings)[0])]
    phraset = [(text[i], result[i]) for i in range(len(text))]
    phraset.sort(key=lambda x: x[1], reverse=True)
    return phraset


def cleaned(t):
    return "".join([i for i in t if i in printable])


def pdf_to_text(pdf):
    with fitz.open(pdf) as doc:
        ina = [cleaned(page.get_text()).split("\n") for page in doc]
        texta = []
        for a in ina:
            texta += a
        print("found text:", texta)
    return [k for k in texta if len(k)>2 and not k.isspace() and k not in bloat]

In [3]:
def process_pdf():
    print("HI")
    context = pdf_to_text('Final_Resumes/Resume_of_ID_0.pdf')
    modified_pdf_files = []
    phraset = phrases_by_relevance(context, ["01/0001 to 02/0002"])
    print('Passing these relevant texts to limiter:', phraset)
    classified = delimiter(phraset)
    print('importance order:', classified)
    print(phraset)
    while True:
        array = []
        while True:
            p = input()
            if p == 'e':
                break
            array.append(p)
        phraset = phrases_by_relevance(context, array)
        print('Passing these relevant texts to limiter:', phraset)
        classified = delimiter(phraset)
        print('importance order:', classified)
        print(phraset)


process_pdf()

HI
found text: ['BILINGUAL LANGUAGE ARTS SIXTH GRADE TEACHER', 'Summary', "Dedicated and enthusiastic professional with over four years' experience in education. Proven expertise in establishing rapport and building trust", 'among students, parents, administrators and community members. Possess strong communication skills and ability to partner across departments', 'within and outside of an organization to meet the needs of students. Motivating students School improvement committee Interactive', 'teaching/learning Interdisciplinary teaching Innovative lesson planning Effectively work with parents', 'Professional Experience', '08/2014 to Current', 'Bilingual Language Arts Sixth Grade Teacher Company Name  City , State', 'Developed and implemented interesting and interactive learning mediums to increase student understanding of course materials and build', 'community within the classroom Participate in the development of intervention plans for students within the classroom setting, durin

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x0 and 1024x60)